# TP2
1. Um  programa imperativo pode ser descrito por um modelo do tipo Control Flow Automaton (CFA) como ilustrado no exemplo seguinte

<img src="automato.png" alt="autómato" style="width: 600px;"/>

Este programa implementa a multiplicação de dois inteiros a,b fornecidos como “input”, e com precisão limitada a $\,n\,$ bits (fornecido como parâmetro do programa). 

Note-se que:

    - Existe a possibilidade de alguma das operações do programa produzir um erro de “overflow”. 
    - Os nós do grafo representam ações  que actuam sobre os “inputs” do nó e produzem um output com as operações indicadas
    - Os ramos do grafo representam ligações que transferem o “output” de um nodo para o “input” do nodo seguinte. Esta transferência é condicionada pela satisfação da condição associada ao ramo


a. Construa um FOTS usando BitVector de tamanho $n$ que descreva o comportamento deste autómato. Para isso identifique as variáveis do modelo, o estado inicial e a relação de transição.

    
b. Verifique se $$\;\;\mathsf{P}\,\equiv\,(x*y + z = a*b)\;\;$$é um invariante deste comportamento.

In [1]:
from pysmt.shortcuts import *
from z3 import *

#### Função declare(t, N):

   A seguinte função cria a $t$-ésima cópia das variáveis de estado, agrupadas num dicionário que nos permite aceder às mesmas pelo nome.

In [2]:
def declare(i, n):
    state = {}
    state['pc'] = Int('pc'+str(i))
    state['x'] = BitVec('x'+str(i), n)
    state['y'] = BitVec('y'+str(i), n)
    state['z'] = BitVec('z'+str(i), n)


    return state


#### Função init(state, prob, N):

    state - Dicionário de variáveis de estado
    a - valor associado ao 1º número a multiplicar
    b - valor associado ao 2º número a multiplicar

   A função `init` tem como objetivo devolver um predicado do Solver que testa se é um possível estado inicial do programa, através do `state`, um dicionário de variáveis.

In [3]:
def init(state, a, b):
    return And(state['pc']== 0, state['x'] == a, state['y'] == b, state['z'] == 0)

#### Função trans(curr, prox, N):

    curr - Estado das variáveis no momento atual
    prox - Estado das variáveis no momento da próxima iteração
    n - Número de bits 
    
   A função `trans` tem como objetivo devolver um predicado do Solver, através dos três estados disponíveis, que teste se é possível transitar entre os estados possíveis.

In [4]:
def trans(curr, prox, n):
    same_values = And(
        prox['x'] == curr['x'],
        prox['y'] == curr['y'],
        prox['z'] == curr['z']    
    )
    
    t0 = And(
        curr['pc'] == 0,
        prox['pc'] == 1,
        same_values
    )
    
    # y = 0
    t1 = And(
        curr['y'] == 0,
        curr['pc'] == 1,
        prox['pc'] == 5,
        same_values
    )
    
    # y != 0 ^ odd(y)
    t2 = And(
        curr['y'] != 0,
        URem(curr['y'], 2) == 1,
        curr['pc'] == 1,
        prox['pc'] == 2,
        same_values
    )
    
    #Q2 -> Q1
    t5 = And(
        curr['pc'] == 2,
        prox['pc'] == 1,
        prox['x'] == curr['x'],
        prox['y'] == curr['y']-1,
        prox['z'] == curr['z'] + curr['x']
    )
    
    # y != 0 ^ even(y)
    t3 = And(
        curr['y'] != 0,
        URem(curr['y'], 2) == 0,
        curr['pc'] == 1,
        prox['pc'] == 3,
        same_values
    )


    # transição em que o solver decide se vai para o estado de overflow ou se continua
    magia = And(
        prox['x'] == curr['x'] << BitVecVal(1, n),
        prox['y'] == curr['y'] >> BitVecVal(1, n),
        prox['z'] == curr['z'],
        
        curr['pc'] == 3,
        
        Or(
            And(UGT(curr['x'], BitVecVal(2**(n-1), n)), prox['pc'] == 1),  # curr['x'] < 2^(n-1)  - não há overflow
            And(ULT(curr['x']  >> BitVecVal(1, n), curr['x']), prox['pc'] == 4),  # curr['x'] >= 2^(n-1) - há overflow
        )
    )
    
    
    # caso de paragem no overflow e no estado final
    stop_case = And(
        prox['pc'] == curr['pc'],
        same_values,
        
        Or(
            And(curr['pc'] == 4, prox['pc'] == 4),
            And(curr['pc'] == 5, prox['pc'] == 5)
        )
    )
    
    
    return Or(t0, t1, t2, t3, t5, stop_case, magia)

#### Função gera_traco(declare, init, trans, k, n, a, b)

    declare - Variáveis de estado
    init - Condições para o estado inicial
    trans - Função transição
    k - Valor do traço
    n - Número de bits
    a - Valor a ser multiplicado
    b - Valor a ser multiplicado

A função `gera_traco` tem como objetivo imprimir o valor das variáveis à medida que vão percorrendo os estados, através das variáveis do estado, de um predicado que testa se um estado é inicial, um número positivo para gerar um possível traço de execução do programa de tamanho `k` , com `N` bits, multiplicando `a` por `b`.

In [6]:
def gera_traco(declare,init,trans,k, n, a, b):

    s = Solver()
    
    
    trace = [declare(i, n) for i in range(k)]

    # adicionar o estado inicial
    s.add(init(trace[0],a,b))
    
    # adicionar as transições
    for i in range(k - 1):
        s.add(trans(trace[i], trace[i+1], n))
    
    
    check = s.check()
    if check == sat:
        m = s.model()
        #print(m)
        #print(trace[i])
        for i in range(k):
            print("Passo ", i)
            for v in trace[i]:
                
                print(v, "=", m[trace[i][v]])
            print("----------------")
    else:
        print(check)
                
gera_traco(declare,init,trans,20, 8, 150, 2)

Passo  0
pc = 0
x = 150
y = 2
z = 0
----------------
Passo  1
pc = 1
x = 150
y = 2
z = 0
----------------
Passo  2
pc = 3
x = 150
y = 2
z = 0
----------------
Passo  3
pc = 4
x = 44
y = 1
z = 0
----------------
Passo  4
pc = 4
x = 44
y = 1
z = 0
----------------
Passo  5
pc = 4
x = 44
y = 1
z = 0
----------------
Passo  6
pc = 4
x = 44
y = 1
z = 0
----------------
Passo  7
pc = 4
x = 44
y = 1
z = 0
----------------
Passo  8
pc = 4
x = 44
y = 1
z = 0
----------------
Passo  9
pc = 4
x = 44
y = 1
z = 0
----------------
Passo  10
pc = 4
x = 44
y = 1
z = 0
----------------
Passo  11
pc = 4
x = 44
y = 1
z = 0
----------------
Passo  12
pc = 4
x = 44
y = 1
z = 0
----------------
Passo  13
pc = 4
x = 44
y = 1
z = 0
----------------
Passo  14
pc = 4
x = 44
y = 1
z = 0
----------------
Passo  15
pc = 4
x = 44
y = 1
z = 0
----------------
Passo  16
pc = 4
x = 44
y = 1
z = 0
----------------
Passo  17
pc = 4
x = 44
y = 1
z = 0
----------------
Passo  18
pc = 4
x = 44
y = 1
z = 0
----------------


In [9]:
gera_traco(declare,init,trans,20, 8, 120, 2)

Passo  0
pc = 0
x = 120
y = 2
z = 0
----------------
Passo  1
pc = 1
x = 120
y = 2
z = 0
----------------
Passo  2
pc = 3
x = 120
y = 2
z = 0
----------------
Passo  3
pc = 1
x = 240
y = 1
z = 0
----------------
Passo  4
pc = 2
x = 240
y = 1
z = 0
----------------
Passo  5
pc = 1
x = 240
y = 0
z = 240
----------------
Passo  6
pc = 5
x = 240
y = 0
z = 240
----------------
Passo  7
pc = 5
x = 240
y = 0
z = 240
----------------
Passo  8
pc = 5
x = 240
y = 0
z = 240
----------------
Passo  9
pc = 5
x = 240
y = 0
z = 240
----------------
Passo  10
pc = 5
x = 240
y = 0
z = 240
----------------
Passo  11
pc = 5
x = 240
y = 0
z = 240
----------------
Passo  12
pc = 5
x = 240
y = 0
z = 240
----------------
Passo  13
pc = 5
x = 240
y = 0
z = 240
----------------
Passo  14
pc = 5
x = 240
y = 0
z = 240
----------------
Passo  15
pc = 5
x = 240
y = 0
z = 240
----------------
Passo  16
pc = 5
x = 240
y = 0
z = 240
----------------
Passo  17
pc = 5
x = 240
y = 0
z = 240
----------------
Passo  18
pc

In [12]:
gera_traco(declare,init,trans,40, 32, 100, 200)

Passo  0
pc = 0
x = 100
y = 200
z = 0
----------------
Passo  1
pc = 1
x = 100
y = 200
z = 0
----------------
Passo  2
pc = 3
x = 100
y = 200
z = 0
----------------
Passo  3
pc = 1
x = 200
y = 100
z = 0
----------------
Passo  4
pc = 3
x = 200
y = 100
z = 0
----------------
Passo  5
pc = 1
x = 400
y = 50
z = 0
----------------
Passo  6
pc = 3
x = 400
y = 50
z = 0
----------------
Passo  7
pc = 1
x = 800
y = 25
z = 0
----------------
Passo  8
pc = 2
x = 800
y = 25
z = 0
----------------
Passo  9
pc = 1
x = 800
y = 24
z = 800
----------------
Passo  10
pc = 3
x = 800
y = 24
z = 800
----------------
Passo  11
pc = 1
x = 1600
y = 12
z = 800
----------------
Passo  12
pc = 3
x = 1600
y = 12
z = 800
----------------
Passo  13
pc = 1
x = 3200
y = 6
z = 800
----------------
Passo  14
pc = 3
x = 3200
y = 6
z = 800
----------------
Passo  15
pc = 1
x = 6400
y = 3
z = 800
----------------
Passo  16
pc = 2
x = 6400
y = 3
z = 800
----------------
Passo  17
pc = 1
x = 6400
y = 2
z = 7200
-----------

In [16]:
def bmc_always(declare,init,trans,inv,K, n):
    for k in range(1,K+1):
        s = Solver()
            
        trace = [declare(i, n) for i in range(k)]

        # adicionar o estado inicial
        s.add(init(trace[0], 150, 2))
        
        for i in range(k - 1):
            s.add(trans(trace[i], trace[i+1], n))
            
        # adicionar a negação do invariante
        #s.add( [Not(inv(trace[i], 150, 2, n) for i in range(k-1))] )
        #print(Not(And([inv(trace[i], 150, 2, n) for i in range(k-1)])))
        #s.add( Or([Not(inv(trace[i], 150, 2, n)) for i in range(k-1)]))
        s.add(Not(And([inv(trace[i], 150, 2, n) for i in range(k-1)])))
        
        check = s.check()
        if check == sat:
            m = s.model()
            for i in range(k):
                print("Passo ", i)
                for v in trace[i]:
                    print(v, "=", m[trace[i][v]])
                print("----------------")
            #return
        else:
            print(check)
        
    print(f"O invariante mantém-se nos primeiros {K} passos")

# invariante -> x*y+z = a*b
def check_inv(state, a, b, n):
    return ( (state['x']*state['y'])+state['z'] == BitVecVal(a, n) * BitVecVal(b, n) )

bmc_always(declare,init,trans,check_inv,20, 8)

unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
unsat
O invariante mantém-se nos primeiros 20 passos


In [17]:
def wrong_inv(state, a, b, n):
    return ( (state['x']*state['y'])+state['z'] == BitVecVal(a, n) * BitVecVal(b, n)*2 )

bmc_always(declare,init,trans,wrong_inv,20, 8)

unsat
Passo  0
pc = 0
x = 150
y = 2
z = 0
----------------
Passo  1
pc = 1
x = 150
y = 2
z = 0
----------------
Passo  0
pc = 0
x = 150
y = 2
z = 0
----------------
Passo  1
pc = 1
x = 150
y = 2
z = 0
----------------
Passo  2
pc = 3
x = 150
y = 2
z = 0
----------------
Passo  0
pc = 0
x = 150
y = 2
z = 0
----------------
Passo  1
pc = 1
x = 150
y = 2
z = 0
----------------
Passo  2
pc = 3
x = 150
y = 2
z = 0
----------------
Passo  3
pc = 4
x = 44
y = 1
z = 0
----------------
Passo  0
pc = 0
x = 150
y = 2
z = 0
----------------
Passo  1
pc = 1
x = 150
y = 2
z = 0
----------------
Passo  2
pc = 3
x = 150
y = 2
z = 0
----------------
Passo  3
pc = 4
x = 44
y = 1
z = 0
----------------
Passo  4
pc = 4
x = 44
y = 1
z = 0
----------------
Passo  0
pc = 0
x = 150
y = 2
z = 0
----------------
Passo  1
pc = 1
x = 150
y = 2
z = 0
----------------
Passo  2
pc = 3
x = 150
y = 2
z = 0
----------------
Passo  3
pc = 4
x = 44
y = 1
z = 0
----------------
Passo  4
pc = 4
x = 44
y = 1
z = 0
---------

pc = 4
x = 44
y = 1
z = 0
----------------
Passo  6
pc = 4
x = 44
y = 1
z = 0
----------------
Passo  7
pc = 4
x = 44
y = 1
z = 0
----------------
Passo  8
pc = 4
x = 44
y = 1
z = 0
----------------
Passo  9
pc = 4
x = 44
y = 1
z = 0
----------------
Passo  10
pc = 4
x = 44
y = 1
z = 0
----------------
Passo  11
pc = 4
x = 44
y = 1
z = 0
----------------
Passo  12
pc = 4
x = 44
y = 1
z = 0
----------------
Passo  13
pc = 4
x = 44
y = 1
z = 0
----------------
Passo  14
pc = 4
x = 44
y = 1
z = 0
----------------
Passo  15
pc = 4
x = 44
y = 1
z = 0
----------------
Passo  16
pc = 4
x = 44
y = 1
z = 0
----------------
Passo  17
pc = 4
x = 44
y = 1
z = 0
----------------
Passo  18
pc = 4
x = 44
y = 1
z = 0
----------------
Passo  19
pc = 4
x = 44
y = 1
z = 0
----------------
O invariante mantém-se nos primeiros 20 passos
